Project 1 Problem 3

In [2]:
using CSV
using DataFrames
using Plots
using LinearAlgebra
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\lane\.julia\packages\Plots\4EfKl\src\backends.jl:373


Plots.PlotlyBackend()

In [3]:
#Problem 3 data

coviddata=CSV.read("C:\\Users\\lane\\Documents\\Julia\\coviddata.csv");
Days=coviddata.Date
I=coviddata.Active
D=coviddata.Deceased
RfI=coviddata.Released_from_Isolation
R=D+RfI
n=length(I);
plot(1:n,I,xrotation = 45, title="Greene County, MO Covid-19 Data", label="Infected", yaxis="Cases", color="red", xtick=(1:7:144,Days[1:7:n]))
plot!(1:n,R, label="Recovered", color="green")

<!DOCTYPE html>
 
 
 Plots.jl

In [4]:
#Data for SIR model

#Total Pop of Greene County MO (2019), source Google
Pop=293086

S=zeros(Int,n)

for i=1:n
  S[i]=Pop-I[i]-R[i]
end

S

144-element Array{Int64,1}:
 293074
 293071
 293068
 293060
 293057
 293054
 293051
 293037
 293034
 293034
 293034
 293033
 293032
      ⋮
 291926
 291889
 291868
 291858
 291800
 291759
 291688
 291655
 291606
 291579
 291550
 291502

**SIR Model for infectious diseases**


The SIR model represents the spread of an infectious disease through a closed population. At any time, each member of the population is either susceptible, infected, or recovered. An infected person becomes recovered once they are no longer contagious. This model assumes that recovered patients cannot be reinfected. 
  
Let $S(t)$ denote the number susceptible, $I(t)$ denote the number infected, and $R(t)$ denote the number recovered at time $t$.  The infection rate, $a$, depends on how contagious the disease is as well as how often people come into contact with each other. The recovery rate, $b$, is the reciprocal of the average length of recovery.
The **SIR model** can be described by the following system of differential equations:

<br>

$$\begin{array}{l}\displaystyle\frac{dS}{dt} = -a S I\\\displaystyle \frac{dI}{dt} = a S I - b I\\\displaystyle \frac{dr}{dt} = b I\end{array}$$

In [8]:
#Estimating Derivatives
dS=zeros(n-1)
for i=1:n-1
  dS[i]=S[i+1]-S[i]
end

dI=zeros(n-1)
for i=1:n-1
  dI[i]=I[i+1]-I[i]
end

dR=zeros(n-1)
for i=1:n-1
  dR[i]=R[i+1]-R[i]
end

# The b matrix in Ax=y
y=[dS;dI;dR]

429-element Array{Float64,1}:
  -3.0
  -3.0
  -8.0
  -3.0
  -3.0
  -3.0
 -14.0
  -3.0
   0.0
   0.0
  -1.0
  -1.0
  -1.0
   ⋮
  25.0
  25.0
   3.0
   3.0
  58.0
  30.0
  42.0
  25.0
  42.0
   3.0
   3.0
  32.0

In [7]:
#Calculate the A matrix in Ax=y

s=zeros(n-1,2)
for i=1:n-1
  s[i,1]=-S[i]*I[i]
end

j=zeros(n-1,2)
for i=1:n-1
  j[i,1]=S[i]*I[i]
  j[i,2]=-I[i]
end

r=zeros(n-1,2)
for i=1:n-1
  r[i,2]=I[i]
end

A=[s;j;r]


429×2 Array{Float64,2}:
 -2.34459e6    0.0
 -3.22378e6    0.0
 -4.10295e6    0.0
 -6.15426e6    0.0
 -5.56808e6    0.0
 -6.15413e6    0.0
 -7.03322e6    0.0
 -1.08424e7    0.0
 -1.17214e7    0.0
 -1.17214e7    0.0
 -1.08423e7    0.0
 -1.02562e7    0.0
 -1.02561e7    0.0
  ⋮          
  0.0        480.0
  0.0        487.0
  0.0        499.0
  0.0        517.0
  0.0        524.0
  0.0        524.0
  0.0        535.0
  0.0        564.0
  0.0        572.0
  0.0        579.0
  0.0        603.0
  0.0        629.0

Part a

In [9]:
#solving for a and b
x=(A'A)\(A'y)

a=x[1]
b=x[2]

0.050452070762135176

Part b

In [10]:
function sir(a,b,x0,timesteps)
  #x0=[S0,I0,R0], initial values of Susceptible,Infected,Recovered
  #a=infection rate, b=recovery rate
  
  x=zeros(3,timesteps+1)
  x[:,1]=x0
  for i=1:timesteps
    x[1,i+1]=x[1,i]-a*x[1,i]*x[2,i]
    x[2,i+1]=x[2,i]+a*x[1,i]*x[2,i]-b*x[2,i]
    x[3,i+1]=x[3,i]+b*x[2,i]
  end
  p=plot!(1:timesteps+1,x[2,:],color="red", label="Infected")
  plot!(1:timesteps+1,x[3,:],color="green", label="Recovered")
  display(p)
  return x
end

x0=[S[1],I[1],R[1]]
sir(a,b,x0,n)

<!DOCTYPE html>
 
 
 Plots.jl

3×145 Array{Float64,2}:
 293074.0  293073.0      293073.0      …  291684.0    291642.0    291598.0
      8.0       8.2398        8.48679        524.163     539.675     555.64
      4.0       4.40362       4.81933        878.246     904.691     931.919

Part c

In [11]:
function sir2(a,b,x0,timesteps)
  #x0=[S0,I0,R0], initial values of Susceptible,Infected,Recovered
  #a=infection rate, b=recovery rate
  
  x=zeros(3,timesteps+1)
  x[:,1]=x0
  for i=1:timesteps
    x[1,i+1]=x[1,i]-a*x[1,i]*x[2,i]
    x[2,i+1]=x[2,i]+a*x[1,i]*x[2,i]-b*x[2,i]
    x[3,i+1]=x[3,i]+b*x[2,i]
  end
  p=plot(1:timesteps+1,x[2,:],color="red", label="Infected")
  plot!(1:timesteps+1,x[3,:],color="green", label="Recovered")
  display(p)
  return x
end
k=sir2(a,b,x0,1000)

<!DOCTYPE html>
 
 
 Plots.jl

3×1001 Array{Float64,2}:
 293074.0  293073.0      293073.0      …  105296.0        105296.0
      8.0       8.2398        8.48679          0.0311411       0.0304698
      4.0       4.40362       4.81933     187790.0        187790.0

Part d

In [12]:
findmax(k[2,:])

#max of I(t)=23693 at t=325


(23693.14023688523, 325)

Part e

In [65]:
#total who will be infected is I(1000)+R(1000) (those who recover/die were technically infected at one point)
total=k[2,1001]+k[3,1001]

# Percentage
100(total/Pop)

#Aproximately 64% will be infected in the long run

64.0734930772393